In [62]:
import pandas as pd
import numpy as np
import helper
import pnf
import pytz
import warnings
warnings.filterwarnings('ignore')
    
etf_panel = helper.get_sector_etf_panel()

# Create a Yearly Slice of ETF Data
Using the ETF data loaded from Alpha Vantage, create a slice that only contains data for a specified date range

In [63]:
# Create copy of etf_dfs for backup
master = etf_panel.copy()
master

<class 'pandas.core.panel.Panel'>
Dimensions: 9 (items) x 5073 (major_axis) x 5 (minor_axis)
Items axis: XLF to XLB
Major_axis axis: 1998-12-22 00:00:00+00:00 to 2019-02-21 00:00:00+00:00
Minor_axis axis: Open to Volume

In [64]:
# Slice panel to only include data between start date and end date
panel_2017 = master.copy()
start_date = '2017-01-01'
end_date = '2017-12-31'

for symbol in panel_2017.items:
    panel_2017[symbol] = panel_2017[symbol][start_date : end_date]
    
# All other dates will have NaNs, so drop those rows
panel_2017 = panel_2017.dropna(axis=1)

# Create RS Matrix for Sector ETFs
Using the sector ETF data, create an RS matrix to compare the sectors to each other

In [65]:
pnf.run_rs_matrix(panel_2017)

,x_count,buy_count,total,XLF,XLV,XLU,XLP,XLI,XLE,XLK,XLY,XLB
XLF,5,7,12,NaN,x_buy,x_buy,x_buy,o_buy,o_buy,x_buy,o_buy,x_sell
XLV,1,2,3,o_sell,NaN,x_buy,o_buy,o_sell,o_sell,o_sell,o_sell,o_sell
XLU,0,0,0,o_sell,o_sell,NaN,o_sell,o_sell,o_sell,o_sell,o_sell,o_sell
XLP,2,3,5,o_sell,x_buy,x_buy,NaN,o_sell,o_sell,o_sell,o_sell,o_buy
XLI,3,4,7,o_sell,x_buy,x_buy,o_buy,NaN,o_sell,o_sell,o_buy,x_sell
XLE,8,5,13,x_sell,x_buy,x_buy,x_buy,x_buy,NaN,x_buy,x_sell,x_sell
XLK,5,6,11,x_sell,x_buy,x_buy,o_buy,o_buy,o_sell,NaN,x_buy,x_buy
XLY,6,4,10,o_sell,x_buy,x_buy,x_buy,x_sell,o_buy,x_sell,NaN,x_sell
XLB,4,5,9,o_sell,x_buy,x_buy,x_buy,o_buy,o_sell,x_sell,o_buy,NaN


# Create RS Charts for Each month
Create an RS chart for each month to be used in yearly aggregations

In [66]:
date_ranges = {
    'Jan':('2017-01-01', '2017-01-31'),
    'Feb':('2017-02-01', '2017-02-28'),
    'Mar':('2017-03-01', '2017-03-31'),
    'Apr':('2017-04-01', '2017-04-30'),
    'May':('2017-05-01', '2017-05-31'),
    'Jun':('2017-06-01', '2017-06-30'),
    'Jul':('2017-07-01', '2017-07-31'),
    'Aug':('2017-08-01', '2017-08-31'),
    'Sep':('2017-09-01', '2017-09-30'),
    'Oct':('2017-10-01', '2017-10-31'),
    'Nov':('2017-11-01', '2017-11-30'),
    'Dec':('2017-12-01', '2017-12-31')    
}

rs_matrices = {}
for month in months:
    # Create panel for month
    start_date = date_ranges[month][0]
    end_date = date_ranges[month][1]
    month_panel = panel_2017.copy()
    for symbol in list(month_panel.items):
        month_panel[symbol] = month_panel[symbol][start_date : end_date]
    month_panel = month_panel.dropna(axis=1)

    # Run RS matrix on this month's panel
    rs_matrices[month] = pnf.run_rs_matrix(month_panel)
    

In [67]:
# Iterate through RS Matrices and tally up yearly total counts
yearly_counts = pd.DataFrame(columns=['x_count', 'buy_count', 'total'], index=monthly_df.columns)
yearly_counts = yearly_counts.fillna(0)

for key in list(rs_matrices.keys()):
    yearly_counts['x_count'] += rs_matrices[key]['x_count']
    yearly_counts['buy_count'] += rs_matrices[key]['buy_count']
    yearly_counts['total'] += rs_matrices[key]['total']
    
yearly_counts

,x_count,buy_count,total
XLF,58,13,71
XLV,53,5,58
XLU,49,9,58
XLP,44,1,45
XLI,65,5,70
XLE,35,9,44
XLK,69,6,75
XLY,68,2,70
XLB,64,7,71
